# LU decomposition

| Who  | Mail | Date | What |
| ---  | ---   | --- | ---      |
|Diego Andrés Alvarez Marín | <daalvarez@unal.edu.co>  | March 20th, 2025 | Initial code |

LU decomposition is a fundamental matrix factorization technique in numerical linear algebra. It decomposes a matrix $\boldsymbol{A}$ into the product of a lower triangular matrix $\boldsymbol{L}$ and an upper triangular matrix $\boldsymbol{U}$:

$$\boldsymbol{A} = \boldsymbol{L}\boldsymbol{U}$$

This decomposition is particularly useful for solving systems of linear equations, computing determinants, and calculating matrix inverses efficiently.

In [1]:
from IPython.display import display
import pandas as pd
import scipy
import numpy as np
from numpy.random import rand
from numpy.linalg import det
import matplotlib.pyplot as plt
import time

# Increase the number of digits shown when an array is printed
np.set_printoptions(linewidth=200)

## 1. Mathematical Background

Given a square matrix $\boldsymbol{A}$, the LU decomposition finds matrices $\boldsymbol{L}$ and $\boldsymbol{U}$ such that:

$$\boldsymbol{A} = \boldsymbol{L}\boldsymbol{U}$$

Where:
- $\boldsymbol{L}$ is a lower triangular matrix with 1's on the diagonal
- $\boldsymbol{U}$ is an upper triangular matrix

For a $3 \times 3$ matrix, the decomposition looks like:

$$
\begin{pmatrix} 
A_{11} & A_{12} & A_{13} \\
A_{21} & A_{22} & A_{23} \\
A_{31} & A_{32} & A_{33}
\end{pmatrix} = 
\begin{pmatrix} 
1 & 0 & 0 \\
L_{21} & 1 & 0 \\
L_{31} & L_{32} & 1
\end{pmatrix}
\begin{pmatrix} 
U_{11} & U_{12} & U_{13} \\
0 & U_{22} & U_{23} \\
0 & 0 & U_{33}
\end{pmatrix}
$$

### Algorithm for LU Decomposition (Doolittle's Method)

The Doolittle algorithm computes $\boldsymbol{L}$ and $\boldsymbol{U}$ directly:

For $i = 1, 2, ..., n$:
   - For $j = i, i+1, ..., n$:
     $$U_{ij} = A_{ij} - \sum_{k=1}^{i-1} L_{ik} U_{kj}$$
   
   - For $j = i+1, i+2, ..., n$:
     $$L_{ji} = \frac{A_{ji} - \sum_{k=1}^{i-1} L_{jk} U_{ki}}{U_{ii}}$$

## 2. Implementation of the LU decomposition without pivoting

In [2]:
def lu_decomposition(A):
    """
    Perform LU decomposition on matrix A using Doolittle's method.
    
    Parameters:
        A (numpy.ndarray): Input square matrix
        
    Returns:
        L (numpy.ndarray): Lower triangular matrix
        U (numpy.ndarray): Upper triangular matrix
    """
    # Check if A is square
    n, m = A.shape
    if n != m:
        raise ValueError("Matrix A must be square")
       
    # Initialize L and U matrices
    L = np.eye(n)
    U = np.zeros((n, n))
    
    for i in range(n):
        # Calculate U[i,j]
        for j in range(i, n):
            U[i, j] = A[i, j] - np.sum(L[i, :i] * U[:i, j])
        
        # Calculate L[j,i]
        for j in range(i+1, n):
            L[j, i] = (A[j, i] - np.sum(L[j, :i] * U[:i, i])) / U[i, i]
    
    return L, U

## 3. Example

Let's test our implementation:

In [3]:
# Create a symmetric positive definite matrix
n = 4
A = rand(n, n)

# Compute Cholesky decomposition
L, U = lu_decomposition(A)

print("Original matrix A:")
print(A)

print("\nLower Triangular Matrix (L2):")
print(L)

print("\nUpper Triangular Matrix (U2):")
print(U)

# Verify the decomposition
print("\nVerify A = L@U:", np.allclose(A, L@U))

Original matrix A:
[[0.92622989 0.68427807 0.03151357 0.19838242]
 [0.15381651 0.89637113 0.04478718 0.94952872]
 [0.08925374 0.33094124 0.50386912 0.86255907]
 [0.962807   0.44670865 0.30044284 0.45587798]]

Lower Triangular Matrix (L2):
[[ 1.          0.          0.          0.        ]
 [ 0.16606732  1.          0.          0.        ]
 [ 0.0963624   0.33855979  1.          0.        ]
 [ 1.03949031 -0.33803498  0.5765935   1.        ]]

Upper Triangular Matrix (U2):
[[0.92622989 0.68427807 0.03151357 0.19838242]
 [0.         0.78273491 0.03955381 0.91658388]
 [0.         0.         0.48744107 0.53312402]
 [0.         0.         0.         0.25210294]]

Verify A = L@U: True


## 2. Implementation of the LU decomposition with pivoting

In [4]:
def lu_pivot(A):
    """Compute LU decomposition of A with partial pivoting using NumPy."""
    n = A.shape[0]
    L = np.eye(n)
    U = A.copy().astype(float)
    P = np.eye(n)

    for i in range(n):
        # Find the pivot element in the i-th column
        pivot_index = np.argmax(np.abs(U[i:, i])) + i

        # Swap rows if necessary
        if pivot_index != i:
            U[[i, pivot_index]] = U[[pivot_index, i]]
            P[[i, pivot_index]] = P[[pivot_index, i]]
            L[[i, pivot_index], :i] = L[[pivot_index, i], :i] # Swap elements in L up to the current column

        # Perform elimination
        if U[i, i] == 0:
            raise ValueError("Pivot element is zero; matrix is singular.")
        for j in range(i + 1, n):
            factor = U[j, i] / U[i, i]
            L[j, i] = factor
            U[j, i:] = U[j, i:] - factor * U[i, i:]

    return P, L, U

In [5]:
# Define a sample square matrix
A = np.array([[2, 1, 1],
              [4, -6, 0],
              [-2, 7, 2]])

print("Original Matrix A:\n", A)

# Perform LU decomposition using our corrected NumPy implementation
try:
    P_np, L_np, U_np = lu_pivot(A.copy())
    print("\nNumPy LU Decomposition (with pivoting):")
    print("Permutation matrix P_np:\n", P_np)
    print("Lower triangular matrix L_np:\n", L_np)
    print("Upper triangular matrix U_np:\n", U_np)
    print("Verification: P_np @ A (should be equal to L_np @ U_np):\n", P_np @ A)
    print("Verification: L_np @ U_np:\n", L_np @ U_np)
except ValueError as e:
    print(f"Error in NumPy LU: {e}")

# Perform LU decomposition using SciPy's lu function
P_scipy, L_scipy, U_scipy = scipy.linalg.lu(A)
print("\nSciPy LU Decomposition (with pivoting):")
print("Permutation matrix P_scipy:\n", P_scipy)
print("Lower triangular matrix L_scipy:\n", L_scipy)
print("Upper triangular matrix U_scipy:\n", U_scipy)
print("Verification: P_scipy @ A (should be equal to L_scipy @ U_scipy):\n", P_scipy @ A)
print("Verification: L_scipy @ U_scipy:\n", L_scipy @ U_scipy)

# Compare the results
print("\nComparison:")
print("Are P_np and P_scipy equal?", np.allclose(P_np, P_scipy))
print("Are L_np and L_scipy equal?", np.allclose(L_np, L_scipy))
print("Are U_np and U_scipy equal?", np.allclose(U_np, U_scipy))

Original Matrix A:
 [[ 2  1  1]
 [ 4 -6  0]
 [-2  7  2]]

NumPy LU Decomposition (with pivoting):
Permutation matrix P_np:
 [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
Lower triangular matrix L_np:
 [[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [-0.5  1.   1. ]]
Upper triangular matrix U_np:
 [[ 4. -6.  0.]
 [ 0.  4.  1.]
 [ 0.  0.  1.]]
Verification: P_np @ A (should be equal to L_np @ U_np):
 [[ 4. -6.  0.]
 [ 2.  1.  1.]
 [-2.  7.  2.]]
Verification: L_np @ U_np:
 [[ 4. -6.  0.]
 [ 2.  1.  1.]
 [-2.  7.  2.]]

SciPy LU Decomposition (with pivoting):
Permutation matrix P_scipy:
 [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
Lower triangular matrix L_scipy:
 [[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [-0.5  1.   1. ]]
Upper triangular matrix U_scipy:
 [[ 4. -6.  0.]
 [ 0.  4.  1.]
 [ 0.  0.  1.]]
Verification: P_scipy @ A (should be equal to L_scipy @ U_scipy):
 [[ 4. -6.  0.]
 [ 2.  1.  1.]
 [-2.  7.  2.]]
Verification: L_scipy @ U_scipy:
 [[ 4. -6.  0.]
 [ 2.  1.  1.]
 [-2.  7.  2.]]

Comparison:
Are P_np and P_s

## 4. Comparison with SciPy's Implementation

Scipy's implementation of the LU decomposition is not compatible with ours, as we will show the following:

In [6]:
P, L, U = lu_pivot(A)

# SciPy's implementation
P_scipy, L_scipy, U_scipy = scipy.linalg.lu(A)

print("SciPy's P:")
print(P_scipy)

print("SciPy's L:")
print(L_scipy)

print("SciPy's U:")
print(U_scipy)

print("\nIs our implementation close to NumPy's?", np.allclose(P, P_scipy) and np.allclose(L, L_scipy) and np.allclose(U, U_scipy))

# Verify the decomposition
print("\nVerify A = P@L@U:", np.allclose(A, P_scipy@L_scipy@U_scipy))

SciPy's P:
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
SciPy's L:
[[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [-0.5  1.   1. ]]
SciPy's U:
[[ 4. -6.  0.]
 [ 0.  4.  1.]
 [ 0.  0.  1.]]

Is our implementation close to NumPy's? True

Verify A = P@L@U: True


## 5. Solving linear systems using LU decomposition

One of the main applications of the LU decomposition is solving linear systems of the form $\boldsymbol{A}\boldsymbol{x} = \boldsymbol{b}$.

Given $\boldsymbol{A} = \boldsymbol{L}\boldsymbol{U}$, we can solve $\boldsymbol{A}\boldsymbol{x} = \boldsymbol{b}$ by:
1. Solving $\boldsymbol{L}\boldsymbol{y} = \boldsymbol{b}$ for $\boldsymbol{y}$
2. Solving $\boldsymbol{U}\boldsymbol{x} = \boldsymbol{y}$ for $\boldsymbol{x}$

Let's implement this:

In [7]:
# Forward substitution to solve Ly = b for y
def forward_substitution(L, b):
    n = len(b)
    y = np.zeros(n)
    for i in range(n):
        y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]
    return y

# Backward substitution to solve Ux = y for x
def backward_substitution(U, y):
    n = len(y)
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    return x

def solve_using_lu(A, b):
    """
    Solve the linear system Ax = b using LU decomposition.
    
    Parameters:
    A (numpy.ndarray): Symmetric positive definite matrix
    b (numpy.ndarray): Right-hand side vector
    
    Returns:
    x (numpy.ndarray): Solution vector
    """
    n = A.shape[0]
    
    # Compute the LU decomposition
    L, U = lu_decomposition(A)
    
    # Forward substitution to solve Ly = b for y
    y = forward_substitution(L, b)

    # Backward substitution to solve Ux = y for x
    x = backward_substitution(U, y)
    
    return x

Let's test our solver with a simple example

In [8]:
# Create a random right-hand side vector
b = rand(n)

# Solve the system using our implementation
x_our = solve_using_lu(A, b)

# Solve the system using NumPy
x_numpy = np.linalg.solve(A, b)

print("Our solution x:")
print(x_our)
print("\nNumPy's solution x:")
print(x_numpy)
print("\nIs our solution close to NumPy's?", np.allclose(x_our, x_numpy))

# Verify the solution
print("\nVerify A@x = b:", np.allclose(A@x_our, b))

IndexError: index 3 is out of bounds for axis 0 with size 3

## 6. Large System Performance

Now, let's compare the performance of our implementation with NumPy's for matrices of different sizes:

In [ ]:
def compare_performance(sizes):
    """
    Compare the performance of our LU implementation with SciPy's.
    """
    lu_times = []
    sp_times = []

    # Create a seed for reproducibility
    np.random.seed(42)
    
    for n in sizes:
        # Create a random matrix    
        A = np.random.randn(n, n)
        
        # Time our implementation
        start_time = time.time()
        L_our, U_our = lu_decomposition(A)
        lu_time = time.time() - start_time
        lu_times.append(lu_time)
        
        # Time NumPy's implementation
        start_time = time.time()
        L_scipy, U_scipy = scipy.linalg.lu(A, permute_l=True)
        sp_time = time.time() - start_time
        sp_times.append(sp_time)

    # Pack all stats in a pandas' dataframe
    df = pd.DataFrame({
        'Size':            sizes,
        'Our time':        lu_times,
        'SciPy time':      sp_times
    })
    return df

In [ ]:
# Compare performance for matrices of different sizes
print("\n\nPerformance Comparison:")
sizes = [10, 20, 50, 100, 200, 500, 1000, 2000]
df = compare_performance(sizes)
display(df)

In [ ]:
# Plot the performance comparison
lu_times = df['Our time']
sp_times = df['SciPy time']

plt.figure(figsize=(10, 6))
plt.plot(sizes, lu_times, 'o-', label='Our LU Implementation')
plt.plot(sizes, sp_times, 's-', label='SciPy Implementation')
plt.xlabel('Matrix Size (n)')
plt.ylabel('Time (seconds)')
plt.title('Performance Comparison: LU vs SciPy')
plt.legend()
plt.grid(True)
plt.xscale('log')
plt.yscale('log')

#k, log_a = np.polyfit(np.log(sizes), np.log(ch_times), 1)
#plt.plot(sizes, np.exp(log_a)*sizes**k)

plt.show()

When data follows a straight line in a log-log plot, it indicates that the relationship between the two variables adheres to a *power law*. Specifically, the data can be modeled by an equation of the form $y = ax^k$. To show that, take the logarithm of both sides of the power law equation transforms it into:
$$
   \log(y) = \log(a) + k\log(x)
$$
This is a linear equation where $k$ is the slope of the line and $\log(a)$ is the intercept.

In [ ]:
k, log_a = np.polyfit(np.log(sizes), np.log(lu_times), 1)
print("k =", k)
print("a =", np.exp(log_a))

The computational complexity of the Cholesky decomposition is $O(n^3/3)$ for an $n \times n$ matrix, which is about 3 times faster than LU decomposition.

## 7. Computing the determinant

Another application of LU decomposition is efficiently computing the determinant of a matrix. For a square matrix $\boldsymbol{A}$ with LU decomposition $\boldsymbol{A} = \boldsymbol{L}\boldsymbol{U}$, the determinant is:

$$\det(\boldsymbol{A}) = \det(\boldsymbol{L}) \cdot \det(\boldsymbol{U}) = \det(\boldsymbol{U}) = \prod_{i=1}^n U_{ii}$$

This is because $\det(\boldsymbol{L}) = 1$ due to its unit diagonal.

In [ ]:
def determinant_using_lu(A):
    """
    Compute the determinant of a matrix using LU decomposition.
    
    Parameters:
        A (numpy.ndarray): Input square matrix
        
    Returns:
        det (float): Determinant of the matrix
    """
    L, U = lu_decomposition(A)
    det_U = np.prod(np.diag(U))
    return det_U

In [ ]:
# Calculate determinants of our example matrices
det_A_lu = determinant_using_lu(A)
det_A_np = det(A)

print(f"Determinant of A using LU decomposition: {det_A_lu:.6f}")
print(f"Determinant of A using NumPy: {det_A_np:.6f}")

## 8. Computing the inverse

LU decomposition can also be used to compute the inverse of a matrix efficiently:

In [ ]:
def inverse_using_lu(A):
    """
    Compute the inverse of a matrix using LU decomposition.
    
    Parameters:
        A (numpy.ndarray): Input square matrix
        
    Returns:
        A_inv (numpy.ndarray): Inverse of the matrix
    """
    n = A.shape[0]
    
    # Perform LU decomposition
    L, U = lu_decomposition(A)
    
    # Initialize inverse matrix
    A_inv = np.zeros((n, n))
    
    # Identity matrix for solving Ax = I
    I = np.eye(n)
    
    # Solve for each column of the identity matrix
    for i in range(n):
        y = forward_substitution(L, I[:,i])
        x = backward_substitution(U, y)
        A_inv[:,i] = x
    
    return A_inv

In [ ]:
# Calculate inverse of our example matrix
A_inv_lu = inverse_using_lu(A)
A_inv_np = np.linalg.inv(A)

print("Original matrix:")
print(A)

print("\nInverse using LU decomposition:")
print(A_inv_lu)

print("\nInverse using NumPy:")
print(A_inv_np)

# Verify the solution
print("\nVerify A*A^{-1} = I:", np.allclose(A@A_inv_lu, np.identity(n)))

## 9. Numerical stability and pivoting

The standard LU decomposition algorithm can encounter numerical instability if small or zero diagonal elements are encountered. Partial pivoting can improve stability by swapping rows to bring the largest element in the column to the diagonal position.

## 10. Conclusions

LU decomposition is a powerful technique for matrix operations, particularly in solving linear systems. It offers computational efficiency for operations like solving multiple systems with the same coefficient matrix or computing matrix inverses.

The main advantages of LU decomposition include:
- Only one decomposition needed to solve multiple systems with the same coefficient matrix
- Efficient calculation of determinants and inverses
- Reduced computational complexity compared to direct methods for large systems

The main challenges relate to numerical stability, which can be mitigated through pivoting strategies. Modern implementations typically use partial or complete pivoting to ensure stable computations.

## To do
Learn how to use the SciPy functions: `lu_factor` and `lu_solve`.